# EDA

## Importar el dataset

In [2]:
# Your code here

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import f_classif, SelectKBest
import pickle

In [3]:
total_data = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/regularized-linear-regression-project-tutorial/main/demographic_health_data.csv", sep = ",")
total_data.to_csv("../data/raw/total_data.csv", index = False)

print(total_data.shape)

total_data.info()

(3140, 108)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3140 entries, 0 to 3139
Columns: 108 entries, fips to Urban_rural_code
dtypes: float64(61), int64(45), object(2)
memory usage: 2.6+ MB


In [4]:
total_data.head()

,fips,TOT_POP,0-9,0-9 y/o % of total pop,19-Oct,10-19 y/o % of total pop,20-29,20-29 y/o % of total pop,30-39,30-39 y/o % of total pop,...,COPD_number,diabetes_prevalence,diabetes_Lower 95% CI,diabetes_Upper 95% CI,diabetes_number,CKD_prevalence,CKD_Lower 95% CI,CKD_Upper 95% CI,CKD_number,Urban_rural_code
0,1001,55601,6787,12.206615,7637,13.735364,6878,12.370281,7089,12.749771,...,3644,12.9,11.9,13.8,5462,3.1,2.9,3.3,1326,3
1,1003,218022,24757,11.355276,26913,12.344167,23579,10.814964,25213,11.564429,...,14692,12.0,11.0,13.1,20520,3.2,3.0,3.5,5479,4
2,1005,24881,2732,10.980266,2960,11.896628,3268,13.134520,3201,12.865239,...,2373,19.7,18.6,20.6,3870,4.5,4.2,4.8,887,6
3,1007,22400,2456,10.964286,2596,11.589286,3029,13.522321,3113,13.897321,...,1789,14.1,13.2,14.9,2511,3.3,3.1,3.6,595,2
4,1009,57840,7095,12.266598,7570,13.087828,6742,11.656293,6884,11.901798,...,4661,13.5,12.6,14.5,6017,3.4,3.2,3.7,1507,2


In [5]:
total_data = total_data.drop_duplicates().reset_index(drop = True)

In [6]:
object_columns = total_data.select_dtypes(include='object').columns
for col in object_columns:
    print(f"\nColumna: {col}")
    


Columna: COUNTY_NAME

Columna: STATE_NAME


In [7]:
total_data["COUNTY_NAME_N"] = pd.factorize(total_data["COUNTY_NAME"])[0]
total_data["STATE_NAME_N"] = pd.factorize(total_data["STATE_NAME"])[0]

In [8]:
columnas = [
    ("COUNTY_NAME_N", "COUNTY_NAME"),
    ("STATE_NAME_N", "STATE_NAME"),
]

transformation_rules = {}

for original_col, normalized_col in columnas:
    mapping = {
        row[original_col]: row[normalized_col]
        for _, row in total_data[[original_col, normalized_col]].drop_duplicates().iterrows()
    }
    transformation_rules[original_col] = mapping


with open("../models/transformation_rules.json", "w") as f:
    json.dump(transformation_rules, f, indent=4)

In [9]:
numeric_columns = total_data.select_dtypes(include=['number']).columns

numeric_columns

Index(['fips', 'TOT_POP', '0-9', '0-9 y/o % of total pop', '19-Oct',
       '10-19 y/o % of total pop', '20-29', '20-29 y/o % of total pop',
       '30-39', '30-39 y/o % of total pop',
       ...
       'diabetes_Lower 95% CI', 'diabetes_Upper 95% CI', 'diabetes_number',
       'CKD_prevalence', 'CKD_Lower 95% CI', 'CKD_Upper 95% CI', 'CKD_number',
       'Urban_rural_code', 'COUNTY_NAME_N', 'STATE_NAME_N'],
      dtype='object', length=108)

In [10]:
target = "Heart disease_number"

numeric_df = total_data.select_dtypes(include=['number'])

corr_matrix = numeric_df.corr().abs()

mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
corr_matrix_masked = corr_matrix.where(~mask)

to_drop = set()

for col in corr_matrix_masked.columns:
    for row in corr_matrix_masked.index:
        if corr_matrix_masked.loc[row, col] > 0.9:
            if col != target and row != target:
                to_drop.add(col)

print(f"Columnas a eliminar por alta correlación (excluyendo '{target}'):")
print(list(to_drop))



Columnas a eliminar por alta correlación (excluyendo 'Heart disease_number'):
['Total Specialist Physicians (2019)', 'CKD_Lower 95% CI', 'Active General Surgeons per 100000 Population 2018 (AAMC)', 'Obesity_prevalence', 'Total Hospitals (2019)', 'Total Population', 'PCTPOVALL_2018', 'Obesity_number', 'county_pop2018_18 and older', 'diabetes_prevalence', 'White-alone pop', 'diabetes_Lower 95% CI', '70-79', 'CKD_prevalence', 'POVALL_2018', "Some college or associate's degree 2014-18", '50-59', 'Total Active Patient Care Physicians per 100000 Population 2018 (AAMC)', '20-29', 'Internal Medicine Primary Care (2019)', 'CI90LBINC_2018', 'High school diploma only 2014-18', 'anycondition_number', 'fips', 'Civilian_labor_force_2018', '0-9', 'STATE_FIPS', '40-49', '70-79 y/o % of total pop', 'Active Physicians per 100000 Population 2018 (AAMC)', 'Unemployed_2018', '80+', 'PCTPOV017_2018', 'Heart disease_prevalence', '19-Oct', 'Employed_2018', 'ICU Beds_x', "Bachelor's degree or higher 2014-18", 

Para ahorrarnos un EDA muy extenso, buscamos las correlaciones superiores a 0´9 y eliminamos una de las 2 columnas

In [11]:
total_data_reduced = total_data.drop(columns=to_drop)

In [12]:
total_data_reduced.to_csv("../data/raw/total_data_reduced.csv", index = False)

## Ingeniería de características

In [13]:
total_data_reduced.describe()

,0-9 y/o % of total pop,10-19 y/o % of total pop,20-29 y/o % of total pop,30-39 y/o % of total pop,40-49 y/o % of total pop,50-59 y/o % of total pop,80+ y/o % of total pop,% White-alone,Black-alone pop,% Black-alone,...,anycondition_Upper 95% CI,Obesity_Upper 95% CI,Heart disease_Upper 95% CI,Heart disease_number,COPD_Upper 95% CI,CKD_Upper 95% CI,CKD_number,Urban_rural_code,COUNTY_NAME_N,STATE_NAME_N
count,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,3.140000e+03,3140.000000,...,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000
mean,11.871051,12.694609,12.283979,11.751535,11.556685,13.526733,4.733801,84.494109,1.395026e+04,9.342405,...,48.838694,36.594713,9.454395,5597.593312,10.438567,3.710478,2466.234076,4.635350,688.478981,26.253503
std,2.124081,1.815044,3.126297,1.696599,1.372935,1.481692,1.535028,16.348987,5.899787e+04,14.478294,...,6.382210,4.578084,1.920387,15615.285499,2.613920,0.613069,7730.422067,1.510447,541.767733,14.260381
min,0.000000,0.000000,0.000000,6.092789,2.631579,5.316361,0.000000,3.841985,0.000000e+00,0.000000,...,23.500000,16.200000,3.700000,7.000000,4.200000,1.900000,3.000000,1.000000,0.000000,0.000000
25%,10.594639,11.674504,10.496774,10.689322,10.685197,12.738561,3.738648,79.620473,1.420000e+02,0.867069,...,45.000000,34.000000,8.200000,798.500000,8.500000,3.300000,314.750000,3.000000,182.000000,14.000000
50%,11.802727,12.687422,11.772649,11.580861,11.618372,13.621339,4.565338,91.410189,9.050000e+02,2.545440,...,49.300000,37.100000,9.400000,1814.500000,10.300000,3.700000,718.000000,5.000000,564.000000,25.000000
75%,12.951840,13.659282,13.182260,12.639379,12.386083,14.429115,5.484143,95.521727,5.765000e+03,10.852053,...,53.100000,39.525000,10.700000,4376.000000,12.100000,4.100000,1776.250000,6.000000,1129.250000,40.000000
max,25.460677,23.304372,37.570198,22.225129,19.430369,20.454545,22.727273,99.043785,1.236170e+06,86.069762,...,68.200000,51.700000,16.400000,434790.000000,22.100000,6.600000,237766.000000,6.000000,1840.000000,50.000000


### Quitar outliers

In [14]:
total_data_con_outliers = total_data_reduced.copy()
total_data_sin_outliers = total_data_reduced.copy()

def replace_outliers_from_column(column, df):
  column_stats = df[column].describe()
  column_iqr = column_stats["75%"] - column_stats["25%"]
  upper_limit = column_stats["75%"] + 1.5 * column_iqr
  lower_limit = column_stats["25%"] - 1.5 * column_iqr

  lower_limit = float(df[column].min())
  # Remove upper outliers
  df[column] = df[column].apply(lambda x: x if (x <= upper_limit) else upper_limit)
  # Remove lower outliers
  df[column] = df[column].apply(lambda x: x if (x >= lower_limit) else lower_limit)
  return df.copy(), [lower_limit, upper_limit]

outliers_dict = {}
for column in total_data_sin_outliers.select_dtypes(include=['number']).columns:
    total_data_sin_outliers, limits_list = replace_outliers_from_column(column, total_data_sin_outliers)
    outliers_dict[column] = limits_list

with open("../models/outliers_replacement.json", "w") as f:
    json.dump(outliers_dict, f)

In [15]:

total_data_con_outliers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3140 entries, 0 to 3139
Data columns (total 47 columns):
 #   Column                                                                         Non-Null Count  Dtype  
---  ------                                                                         --------------  -----  
 0   0-9 y/o % of total pop                                                         3140 non-null   float64
 1   10-19 y/o % of total pop                                                       3140 non-null   float64
 2   20-29 y/o % of total pop                                                       3140 non-null   float64
 3   30-39 y/o % of total pop                                                       3140 non-null   float64
 4   40-49 y/o % of total pop                                                       3140 non-null   float64
 5   50-59 y/o % of total pop                                                       3140 non-null   float64
 6   80+ y/o % of total pop  

In [16]:
print("Heart disease_number" in total_data_sin_outliers.columns)

True


## Escalado de valores

In [17]:
numeric_columns = total_data_reduced.select_dtypes(include=['number']).columns.drop(target)

X_con_outliers = total_data_con_outliers[numeric_columns]
X_sin_outliers = total_data_sin_outliers[numeric_columns]
y = total_data_con_outliers[target]

X_train_con_outliers, X_test_con_outliers, y_train, y_test = train_test_split(X_con_outliers, y, test_size = 0.2, random_state = 42)
X_train_sin_outliers, X_test_sin_outliers = train_test_split(X_sin_outliers, test_size = 0.2, random_state = 42)


X_train_con_outliers.to_excel("../data/processed/X_train_con_outliers.xlsx", index = False)
X_train_sin_outliers.to_excel("../data/processed/X_train_sin_outliers.xlsx", index = False)
X_test_con_outliers.to_excel("../data/processed/X_test_con_outliers.xlsx", index = False)
X_test_sin_outliers.to_excel("../data/processed/X_test_sin_outliers.xlsx", index = False)
y_train.to_excel("../data/processed/y_train.xlsx", index = False)
y_test.to_excel("../data/processed/y_test.xlsx", index = False)

### Normalización

In [18]:
normalizador_con_outliers = StandardScaler()
normalizador_con_outliers.fit(X_train_con_outliers)

with open("../models/normalizador_con_outliers.pkl", "wb") as file:
    pickle.dump(normalizador_con_outliers,file)

X_train_con_outliers_norm = normalizador_con_outliers.transform(X_train_con_outliers)
X_train_con_outliers_norm = pd.DataFrame(X_train_con_outliers_norm, index = X_train_con_outliers.index, columns = numeric_columns)

X_test_con_outliers_norm = normalizador_con_outliers.transform(X_test_con_outliers)
X_test_con_outliers_norm = pd.DataFrame(X_test_con_outliers_norm, index = X_test_con_outliers.index, columns = numeric_columns)

X_train_con_outliers_norm.to_excel("../data/processed/X_train_con_outliers_norm.xlsx", index = False)
X_test_con_outliers_norm.to_excel("../data/processed/X_test_con_outliers_norm.xlsx", index = False)

normalizador_sin_outliers = StandardScaler()
normalizador_sin_outliers.fit(X_train_sin_outliers)

with open("../models/normalizador_sin_outliers.pkl", "wb") as file:
    pickle.dump(normalizador_sin_outliers,file)

X_train_sin_outliers_norm = normalizador_sin_outliers.transform(X_train_sin_outliers)
X_train_sin_outliers_norm = pd.DataFrame(X_train_sin_outliers_norm, index = X_train_sin_outliers.index, columns = numeric_columns)

X_test_sin_outliers_norm = normalizador_sin_outliers.transform(X_test_sin_outliers)
X_test_sin_outliers_norm = pd.DataFrame(X_test_sin_outliers_norm, index = X_test_sin_outliers.index, columns = numeric_columns)

X_train_sin_outliers_norm.to_excel("../data/processed/X_train_sin_outliers_norm.xlsx", index = False)
X_test_sin_outliers_norm.to_excel("../data/processed/X_test_sin_outliers_norm.xlsx", index = False)

X_train_con_outliers_norm.head()

,0-9 y/o % of total pop,10-19 y/o % of total pop,20-29 y/o % of total pop,30-39 y/o % of total pop,40-49 y/o % of total pop,50-59 y/o % of total pop,80+ y/o % of total pop,% White-alone,Black-alone pop,% Black-alone,...,CNTY_FIPS,anycondition_Upper 95% CI,Obesity_Upper 95% CI,Heart disease_Upper 95% CI,COPD_Upper 95% CI,CKD_Upper 95% CI,CKD_number,Urban_rural_code,COUNTY_NAME_N,STATE_NAME_N
1292,0.102383,0.162374,-0.429454,-0.665485,-0.698327,0.585470,-0.117970,0.648457,-0.229971,-0.557226,...,0.211484,0.484458,0.833508,0.874465,0.989521,0.657452,-0.216950,0.910528,0.343882,-0.317280
2302,-0.754597,-0.836073,-0.736296,-0.785934,-0.073361,1.253363,0.651730,0.759892,-0.225814,-0.564855,...,0.156293,0.061751,-0.324419,0.350729,0.413368,0.161947,-0.135212,0.249092,1.314203,0.812126
761,-0.717144,0.464170,1.320194,-0.128551,0.159743,-0.000813,-0.383537,0.538703,-0.212142,-0.379993,...,0.266675,0.046095,-0.258876,-0.696745,-0.009144,-0.994232,-0.204982,-1.735217,-0.676341,-0.881983
2194,0.120407,0.576280,0.094875,0.335452,0.652423,0.247740,-0.613492,-0.579575,-0.216364,-0.556409,...,0.248278,-0.016528,0.331011,-0.487250,-0.431656,-0.829064,-0.055748,-1.073780,1.214399,0.670950
1241,0.252809,0.234535,0.173362,0.070962,0.159841,-0.099992,-0.148575,-0.191312,0.008182,0.134914,...,-0.726760,0.155686,0.527640,-0.173008,0.221317,-0.168390,0.108446,-0.412344,-1.267775,-0.317280


### Escalado min-max

In [19]:
min_max_con_outliers = MinMaxScaler()
min_max_con_outliers.fit(X_train_con_outliers)

with open("../models/min_max_con_outliers.pkl", "wb") as file:
    pickle.dump(min_max_con_outliers,file)

X_train_con_outliers_scal = min_max_con_outliers.transform(X_train_con_outliers)
X_train_con_outliers_scal = pd.DataFrame(X_train_con_outliers_scal, index = X_train_con_outliers.index, columns = numeric_columns)

X_test_con_outliers_scal = min_max_con_outliers.transform(X_test_con_outliers)
X_test_con_outliers_scal = pd.DataFrame(X_test_con_outliers_scal, index = X_test_con_outliers.index, columns = numeric_columns)

X_train_con_outliers_scal.to_excel("../data/processed/X_train_con_outliers_scal.xlsx", index = False)
X_test_con_outliers_scal.to_excel("../data/processed/X_test_con_outliers_scal.xlsx", index = False)

min_max_sin_outliers = MinMaxScaler()
min_max_sin_outliers.fit(X_train_sin_outliers)

with open("../models/min_max_sin_outliers.pkl", "wb") as file:
    pickle.dump(min_max_sin_outliers,file)

X_train_sin_outliers_scal = min_max_sin_outliers.transform(X_train_sin_outliers)
X_train_sin_outliers_scal = pd.DataFrame(X_train_sin_outliers_scal, index = X_train_sin_outliers.index, columns = numeric_columns)

X_test_sin_outliers_scal = min_max_sin_outliers.transform(X_test_sin_outliers)
X_test_sin_outliers_scal = pd.DataFrame(X_test_sin_outliers_scal, index = X_test_sin_outliers.index, columns = numeric_columns)

X_train_sin_outliers_scal.to_excel("../data/processed/X_train_sin_outliers_scal.xlsx", index = False)
X_test_sin_outliers_scal.to_excel("../data/processed/X_test_sin_outliers_scal.xlsx", index = False)

X_train_con_outliers_scal.head()

,0-9 y/o % of total pop,10-19 y/o % of total pop,20-29 y/o % of total pop,30-39 y/o % of total pop,40-49 y/o % of total pop,50-59 y/o % of total pop,80+ y/o % of total pop,% White-alone,Black-alone pop,% Black-alone,...,CNTY_FIPS,anycondition_Upper 95% CI,Obesity_Upper 95% CI,Heart disease_Upper 95% CI,COPD_Upper 95% CI,CKD_Upper 95% CI,CKD_number,Urban_rural_code,COUNTY_NAME_N,STATE_NAME_N
1292,0.484236,0.558117,0.316717,0.280961,0.487737,0.577360,0.199632,0.957623,0.000273,0.014706,...,0.150179,0.635347,0.681690,0.565574,0.491620,0.456522,0.003230,1.0,0.477693,0.44
2302,0.411625,0.481367,0.289175,0.268121,0.538742,0.646467,0.251301,0.976818,0.000480,0.013439,...,0.143027,0.574944,0.532394,0.483607,0.407821,0.391304,0.006103,0.8,0.763330,0.76
761,0.414798,0.581316,0.473765,0.338200,0.557766,0.516697,0.181805,0.938718,0.001161,0.044132,...,0.157330,0.572707,0.540845,0.319672,0.346369,0.239130,0.003651,0.2,0.177367,0.28
2194,0.485763,0.589934,0.363781,0.387664,0.597975,0.542415,0.166368,0.746091,0.000951,0.014841,...,0.154946,0.563758,0.616901,0.352459,0.284916,0.260870,0.008895,0.4,0.733950,0.72
1241,0.496981,0.563664,0.370826,0.359468,0.557774,0.506435,0.197577,0.812971,0.012137,0.129621,...,0.028605,0.588367,0.642254,0.401639,0.379888,0.347826,0.014666,0.6,0.003264,0.44


## Feature selection

In [20]:
selection_model = SelectKBest(f_classif, k = 10)
selection_model.fit(X_train_con_outliers_scal, y_train)

ix = selection_model.get_support()
X_train_sel = pd.DataFrame(selection_model.transform(X_train_con_outliers_scal), columns = X_train_con_outliers_scal.columns.values[ix])
X_test_sel = pd.DataFrame(selection_model.transform(X_test_con_outliers_scal), columns = X_test_con_outliers_scal.columns.values[ix])

with open("../models/feature_selection_k_10.json", "w") as f:
    json.dump(X_train_sel.columns.tolist(), f)

X_train_sel.head()

,Black-alone pop,Native American/American Indian-alone pop,Asian-alone pop,% Asian-alone,Hawaiian/Pacific Islander-alone pop,% Hawaiian/PI-alone,N_POP_CHG_2018,Percent of adults with a bachelor's degree or higher 2014-18,CKD_number,Urban_rural_code
0,0.000273,0.002947,0.000057,0.007782,0.000117,0.000846,0.229694,0.252011,0.003230,1.0
1,0.000480,0.000750,0.000149,0.010490,0.000234,0.000878,0.222816,0.235925,0.006103,0.8
2,0.001161,0.000929,0.000254,0.024281,0.000191,0.000975,0.229713,0.215818,0.003651,0.2
3,0.000951,0.085388,0.000883,0.034675,0.000840,0.001758,0.232905,0.325737,0.008895,0.4
4,0.012137,0.007208,0.002594,0.069696,0.000648,0.000928,0.229333,0.284182,0.014666,0.6


In [21]:
X_train_sel[target] = list(y_train)
X_test_sel[target] = list(y_test)

X_train_sel.to_csv("../data/processed/clean_train.csv", index=False)
X_test_sel.to_csv("../data/processed/clean_test.csv", index=False)